In [2]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

c:\Users\WONJONGHYEON\anaconda3\envs\CP2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [164]:
CFG = {
    'EPOCHS':10,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':256,
    'SEED':41
}

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

# 데이터 불러오기 및 정리

In [ ]:
!unzip open.zip

In [143]:
df=pd.read_csv('df_aug.csv')
df_copy=df.copy()

In [144]:
df

,문장,유형,극성,시제,확실성
0,비가 내리는 지역에서는 치고 곳이 지역에선 우박이 떨어지는 일부 있을 예정이다,2,0,1,0
1,비가 떨어지는 지역에서는 돌풍과 함께 천둥번개가 치고 일부 지역에선 우박이 내리는 ...,2,0,1,0
2,비가 내리는 지역에서는 돌풍과 함께 천둥번개가 치고 일부 우박이 있을 곳이 떨어지는...,2,0,1,0
3,비가 곳이 지역에서는 천둥번개가 치고 일부 우박이 떨어지는 내리는 예정이다,2,0,1,0
4,비가 내리는 곳이 돌풍과 치고 일부 지역에선 우박이 떨어지는 지역에서는 예정이다,2,0,1,0
...,...,...,...,...,...
28119,또한 a씨가 자신이 근무한 편의점에서 1일 동안 최소 1만원 이상의 돈을 들여 상품...,1,0,0,0
28120,오유리 빗썸경제연구소 정책연구팀장은 국내 규제가 미국의 동향을 따를 가능성이 높은 ...,1,0,0,1
28121,보라네트워크는 카카오게임즈의 계열사다,1,0,2,1
28122,컴투스는 최근 크리티카 글로벌의 a1a 블록체인 생태계 합류를 위해 참여자들이 직접...,1,0,0,1


In [137]:
df=pd.read_csv('train.csv')

In [43]:
df=df.sample(frac=1)

In [138]:
df

,ID,문장,유형,극성,시제,확실성,label
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
...,...,...,...,...,...,...,...
16536,TRAIN_16536,"＇신동덤＇은 ＇신비한 동물사전＇과 ＇해리 포터＇ 시리즈를 잇는 마법 어드벤처물로, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16537,TRAIN_16537,"수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목, 어깨, 팔꿈치, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16538,TRAIN_16538,김금희 소설가는 ＂계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
16539,TRAIN_16539,1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...,사실형,긍정,과거,불확실,사실형-긍정-과거-불확실


In [111]:
train, val= train_test_split(df, test_size=0.2, random_state=CFG['SEED'])

In [112]:
train

,ID,문장,유형,극성,시제,확실성
11351,TRAIN_11372,지방 관리들이 세금을 가로채기 위해 중앙정부에 가능한 실제 인구수를 감추어 신고해 ...,1,0,0,1
10069,TRAIN_10087,중국 정부와 공산당 국민이 이심전심으로 일사불란하게 움직이며 이어가는 전근대적 규제다,1,0,2,1
10902,TRAIN_10922,소득 격차가 줄어든 것이 저소득층이 일해 번 소득이 늘어서가 아니라 고소득층 소득 ...,3,0,2,1
14003,TRAIN_14032,1일부터 1일까지 열리는 서리산의 대왕의 경우 시즌1 에피소드1 얼음동굴편에 등장하...,1,0,0,1
7622,TRAIN_07635,삼성전자가 이번주에만 1주 연속 신저가를 1번 넘게 갈아치우며 가파른 하락세를 타자...,1,0,2,1
...,...,...,...,...,...,...
6819,TRAIN_06829,한국 추상화 거장 이우환이 1년 위작 파동 이후 처음으로 국내 개인전을 연다,1,0,2,1
15829,TRAIN_15861,층수만 본래 1층에서 1층으로 증축됐을 뿐 1여년간 한 자리를 지켜왔다,1,0,0,1
8513,TRAIN_08528,반려동물이 가장 많이 탑승한 노선은 제주였고 특히 부산제주 노선은 전체 운항 편수 ...,1,0,0,1
931,TRAIN_00932,아이리시맨는 그 다음이었다,1,0,0,1


In [166]:
val

,ID,문장,유형,극성,시제,확실성
4020,TRAIN_04025,지난 1일 지효는 a앱 트와이스 채널에서 팬들과 채팅을 하던 중 관종 웅앵웅이라는 ...,1,0,0,1
4366,TRAIN_04372,1년 이 대회에서 토머스는 1홀 최소타 신기록으로 우승한 좋은 기억도 있다,1,0,0,1
604,TRAIN_00605,샤론 최는 한국 국적 보유자로 미국 대학에서 영화를 공부 중인 만큼 봉 감독이 말하...,1,0,2,1
770,TRAIN_00771,개발원 관계자는 인터넷으로 보장 내용을 설계할 경우 사고 발생시 충분한 보상을 받을...,3,0,0,1
3034,TRAIN_03038,또 불포화 지방은 과일과 채소에 있는 카로티노이드 흡수를 도와 암 발생률을 감소시킬...,1,0,1,0
...,...,...,...,...,...,...
3908,TRAIN_03913,실로 오랜만에 주어진 행복을 행여 놓칠세라 카메라에 담는 여행자들의 모습이 정겹다,3,0,2,1
16079,TRAIN_16112,aa의 운용 주체인 공군은 1월 1일부터 오는 1일까지 한국형 전투기 aa 명칭 공...,1,1,1,1
2567,TRAIN_02571,각본상을 받은 뒤에는 시나리오를 쓴다는 게 사실 고독하고 외로운 작업이다 국가를 대...,1,0,0,1
11433,TRAIN_11455,해약 환급금 규모도 급격히 늘어나는 추세다,1,0,2,1


In [8]:
test=pd.read_csv(r'C:\Users\WONJONGHYEON\OneDrive\바탕 화면\cp2-competition\test.csv')
test_copy=test.copy()

In [64]:
test

,ID,문장
0,TEST_0000,"장욱진의 ＇가족＇은 허물 없는 가족애를, 처음 공개되는 정약용의 ＇정효자전＇과 ＇정..."
1,TEST_0001,"조지 W 부시, 버락 오바마 전 대통령도 전쟁 위험 때문에 버린 카드다."
2,TEST_0002,지난해 1분기 128억원이었던 영업이익이 올해 1분기 505억원으로 급증했다.
3,TEST_0003,수상 작가와 맺으려던 계약서 내용 가운데 일부가 ＇독소 조항＇으로 해석돼 수정을 요...
4,TEST_0004,결국 최근 KDB산업은행은 대규모 손실 위기에 닥친 에어부산에 140억원 금융지원을...
...,...,...
7085,TEST_7085,"2020 세계국가편람 모바일 앱은 세계 216개국의 국가개황과 주요 경제지표, 사회..."
7086,TEST_7086,탈세계화 징후들이 반갑지 않은 이유다.
7087,TEST_7087,"틱톡은 6월 ＇인터넷 안전의 달＇을 맞아 올바른 개인정보 보호 관리 방법, 앱 내 ..."
7088,TEST_7088,만약 3개월 간 채굴자들의 투표를 거쳐 2/3 이상의 해시파워가 ＇채굴세＇ 도입에 ...


## 벡터화 및 레이블링

In [ ]:
# 2. Label Encoding (유형, 극성, 시제, 확실성)
type_le = preprocessing.LabelEncoder()
train["유형"] = type_le.fit_transform(train["유형"].values)
val["유형"] = type_le.transform(val["유형"].values)

polarity_le = preprocessing.LabelEncoder()
train["극성"] = polarity_le.fit_transform(train["극성"].values)
val["극성"] = polarity_le.transform(val["극성"].values)

tense_le = preprocessing.LabelEncoder()
train["시제"] = tense_le.fit_transform(train["시제"].values)
val["시제"] = tense_le.transform(val["시제"].values)

certainty_le = preprocessing.LabelEncoder()
train["확실성"] = certainty_le.fit_transform(train["확실성"].values)
val["확실성"] = certainty_le.transform(val["확실성"].values)

In [114]:
vectorizer = TfidfVectorizer(min_df = 4, analyzer = 'word', ngram_range=(1, 2))
vectorizer.fit(np.array(train["문장"]))

train_vec = vectorizer.transform(train["문장"])
val_vec = vectorizer.transform(val["문장"])
test_vec = vectorizer.transform(test["문장"])

print(train_vec.shape, val_vec.shape, test_vec.shape)

(13204, 8889) (3302, 8889) (7090, 8889)


In [115]:
train_type = train["유형"].values # sentence type
train_polarity = train["극성"].values # sentence polarity
train_tense = train["시제"].values # sentence tense
train_certainty = train["확실성"].values # sentence certainty

train_labels = {
    'type' : train_type,
    'polarity' : train_polarity,
    'tense' : train_tense,
    'certainty' : train_certainty
}

In [116]:
val_type = val["유형"].values # sentence type
val_polarity = val["극성"].values # sentence polarity
val_tense = val["시제"].values # sentence tense
val_certainty = val["확실성"].values # sentence certainty

val_labels = {
    'type' : val_type,
    'polarity' : val_polarity,
    'tense' : val_tense,
    'certainty' : val_certainty
}

In [158]:
val_type

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [117]:
class CustomDataset(Dataset):
    def __init__(self, st_vec, st_labels):
        self.st_vec = st_vec
        self.st_labels = st_labels

    def __getitem__(self, index):
        st_vector = torch.FloatTensor(self.st_vec[index].toarray()).squeeze(0)
        if self.st_labels is not None:
            st_type = self.st_labels['type'][index]
            st_polarity = self.st_labels['polarity'][index]
            st_tense = self.st_labels['tense'][index]
            st_certainty = self.st_labels['certainty'][index]
            return st_vector, st_type, st_polarity, st_tense, st_certainty
        else:
            return st_vector

    def __len__(self):
        return len(self.st_vec.toarray())

In [118]:
train_dataset = CustomDataset(train_vec, train_labels)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_vec, val_labels)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [153]:
train_dataset[1]

(tensor([0., 0., 0.,  ..., 0., 0., 0.]), 1, 0, 2, 1)

In [ ]:
next(iter(train_loader))

In [120]:
torch.sum(train_dataset[132][0])

tensor(2.6195)

### 베이스모델

In [172]:
class BaseModel(nn.Module):
    def __init__(self, input_dim=8889):
        super(BaseModel, self).__init__()
        self.feature_extract = nn.Sequential(
            nn.Linear(in_features=input_dim, out_features=1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Linear(in_features=1024, out_features=1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Linear(in_features=1024, out_features=512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
        )
        self.type_classifier = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=512, out_features=4),
        )
        self.polarity_classifier = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=512, out_features=3),
        )
        self.tense_classifier = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=512, out_features=3),
        )
        self.certainty_classifier = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=512, out_features=2),
        )
            
    def forward(self, x):
        x = self.feature_extract(x)
        # 문장 유형, 극성, 시제, 확실성을 각각 분류
        type_output = self.type_classifier(x)
        polarity_output = self.polarity_classifier(x)
        tense_output = self.tense_classifier(x)
        certainty_output = self.certainty_classifier(x)
        return type_output, polarity_output, tense_output, certainty_output

In [173]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    
    criterion = {
        'type' : nn.CrossEntropyLoss().to(device),
        'polarity' : nn.CrossEntropyLoss().to(device),
        'tense' : nn.CrossEntropyLoss().to(device),
        'certainty' : nn.CrossEntropyLoss().to(device)
    }
    
    best_loss = 999999
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for sentence, type_label, polarity_label, tense_label, certainty_label in tqdm(iter(train_loader)):
            sentence = sentence.to(device)
            type_label = type_label.to(device)
            polarity_label = polarity_label.to(device)
            tense_label = tense_label.to(device)
            certainty_label = certainty_label.to(device)
            
            optimizer.zero_grad()
            
            type_logit, polarity_logit, tense_logit, certainty_logit = model(sentence)
            
            loss = 0.25 * criterion['type'](type_logit, type_label) + \
                    0.25 * criterion['polarity'](polarity_logit, polarity_label) + \
                    0.25 * criterion['tense'](tense_logit, tense_label) + \
                    0.25 * criterion['certainty'](certainty_logit, certainty_label)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        val_loss, val_type_f1, val_polarity_f1, val_tense_f1, val_certainty_f1 = validation(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}] 유형 F1 : [{val_type_f1:.5f}] 극성 F1 : [{val_polarity_f1:.5f}] 시제 F1 : [{val_tense_f1:.5f}] 확실성 F1 : [{val_certainty_f1:.5f}]')
        
        if scheduler is not None:
            scheduler.step(val_loss)
            
        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            
    return best_model

In [174]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    
    type_preds, polarity_preds, tense_preds, certainty_preds = [], [], [], []
    type_labels, polarity_labels, tense_labels, certainty_labels = [], [], [], []
    
    
    with torch.no_grad():
        for sentence, type_label, polarity_label, tense_label, certainty_label in tqdm(iter(val_loader)):
            sentence = sentence.to(device)
            type_label = type_label.to(device)
            polarity_label = polarity_label.to(device)
            tense_label = tense_label.to(device)
            certainty_label = certainty_label.to(device)
            
            type_logit, polarity_logit, tense_logit, certainty_logit = model(sentence)
            
            loss = 0.25 * criterion['type'](type_logit, type_label) + \
                    0.25 * criterion['polarity'](polarity_logit, polarity_label) + \
                    0.25 * criterion['tense'](tense_logit, tense_label) + \
                    0.25 * criterion['certainty'](certainty_logit, certainty_label)
            
            val_loss.append(loss.item())
            
            type_preds += type_logit.argmax(1).detach().cpu().numpy().tolist()
            type_labels += type_label.detach().cpu().numpy().tolist()
            
            polarity_preds += polarity_logit.argmax(1).detach().cpu().numpy().tolist()
            polarity_labels += polarity_label.detach().cpu().numpy().tolist()
            
            tense_preds += tense_logit.argmax(1).detach().cpu().numpy().tolist()
            tense_labels += tense_label.detach().cpu().numpy().tolist()
            
            certainty_preds += certainty_logit.argmax(1).detach().cpu().numpy().tolist()
            certainty_labels += certainty_label.detach().cpu().numpy().tolist()
    
    type_f1 = f1_score(type_labels, type_preds, average='weighted')
    polarity_f1 = f1_score(polarity_labels, polarity_preds, average='weighted')
    tense_f1 = f1_score(tense_labels, tense_preds, average='weighted')
    certainty_f1 = f1_score(certainty_labels, certainty_preds, average='weighted')
    
    return np.mean(val_loss), type_f1, polarity_f1, tense_f1, certainty_f1

In [175]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

train(model, optimizer, train_loader, val_loader, scheduler, device)

100%|██████████| 13/13 [00:02<00:00,  4.56it/s]


Epoch : [1] Train Loss : [0.73990] Val Loss : [0.56353] 유형 F1 : [0.73694] 극성 F1 : [0.93820] 시제 F1 : [0.46060] 확실성 F1 : [0.88130]


100%|██████████| 13/13 [00:03<00:00,  3.98it/s]


Epoch : [2] Train Loss : [0.30053] Val Loss : [0.42710] 유형 F1 : [0.77207] 극성 F1 : [0.95047] 시제 F1 : [0.71788] 확실성 F1 : [0.88869]


100%|██████████| 13/13 [00:03<00:00,  4.23it/s]


...

Epoch : [4] Train Loss : [0.08874] Val Loss : [0.40582] 유형 F1 : [0.78788] 극성 F1 : [0.95715] 시제 F1 : [0.72807] 확실성 F1 : [0.89573]


100%|██████████| 13/13 [00:02<00:00,  4.62it/s]


Epoch : [5] Train Loss : [0.05836] Val Loss : [0.41261] 유형 F1 : [0.79129] 극성 F1 : [0.95911] 시제 F1 : [0.72314] 확실성 F1 : [0.89572]


 25%|██▌       | 13/52 [00:07<00:22,  1.71it/s]


KeyboardInterrupt: 

In [160]:
infer_model

BaseModel(
  (feature_extract): Sequential(
    (0): Linear(in_features=8889, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Linear(in_features=1024, out_features=1024, bias=True)
    (4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=1024, out_features=512, bias=True)
    (7): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
  )
  (type_classifier): Sequential(
    (0): Dropout(p=0.3, inplace=False)
    (1): Linear(in_features=512, out_features=4, bias=True)
  )
  (polarity_classifier): Sequential(
    (0): Dropout(p=0.3, inplace=False)
    (1): Linear(in_features=512, out_features=3, bias=True)
  )
  (tense_classifier): Sequential(
    (0): Dropout(p=0.3, inplace=False)
    (1): Linear(in_f

In [125]:
test_dataset = CustomDataset(test_vec, None)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [126]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    type_preds, polarity_preds, tense_preds, certainty_preds = [], [], [], []
    
    with torch.no_grad():
        for sentence in tqdm(test_loader):
            sentence = sentence.to(device)
            
            type_logit, polarity_logit, tense_logit, certainty_logit = model(sentence)
            
            type_preds += type_logit.argmax(1).detach().cpu().numpy().tolist()
            polarity_preds += polarity_logit.argmax(1).detach().cpu().numpy().tolist()
            tense_preds += tense_logit.argmax(1).detach().cpu().numpy().tolist()
            certainty_preds += certainty_logit.argmax(1).detach().cpu().numpy().tolist()
            
    return type_preds, polarity_preds, tense_preds, certainty_preds

In [127]:
type_preds, polarity_preds, tense_preds, certainty_preds = inference(model, test_loader, device)

100%|██████████| 28/28 [00:05<00:00,  5.51it/s]


In [128]:
np.mean(type_preds)

1.1691114245416079

In [ ]:
type_preds = type_le.inverse_transform(type_preds)
polarity_preds = polarity_le.inverse_transform(polarity_preds)
tense_preds = tense_le.inverse_transform(tense_preds)
certainty_preds = certainty_le.inverse_transform(certainty_preds)

In [ ]:
predictions = []
for type_pred, polarity_pred, tense_pred, certainty_pred in zip(type_preds, polarity_preds, tense_preds, certainty_preds):
    predictions.append(type_pred+'-'+polarity_pred+'-'+tense_pred+'-'+certainty_pred)

In [130]:
submit = pd.read_csv(r'C:\Users\WONJONGHYEON\OneDrive\바탕 화면\cp2-competition\sample_submission.csv')

In [132]:

submit['label'] = submit['유형'] + '-' + submit['극성'] + '-' + submit['시제'] + '-' + submit['확실성']

In [135]:
submit

,ID,label
0,TEST_0000,사실형-긍정-과거-확실
1,TEST_0001,사실형-긍정-현재-확실
2,TEST_0002,사실형-긍정-과거-확실
3,TEST_0003,사실형-긍정-과거-확실
4,TEST_0004,사실형-긍정-과거-확실
...,...,...
7085,TEST_7085,사실형-긍정-현재-확실
7086,TEST_7086,추론형-긍정-현재-확실
7087,TEST_7087,사실형-긍정-현재-확실
7088,TEST_7088,사실형-긍정-현재-확실


In [136]:
submit.to_csv('submit_bs2.csv', index=False, encoding='utf-8-sig')